In [114]:
import random
import csv
import pandas as pd
import tenseal as ts
import numpy as np
import ast

"""
Library used for encryption:
https://github.com/OpenMined/TenSEAL
Visit for installation instructions
"""

In [108]:
def generate_minutiae(num_fingerprints, num_minutiae_per_fingerprint):
    dataset = []
    for i in range(num_fingerprints):
        fingerprint = []
        fingerprint_id = i+1
        for _ in range(num_minutiae_per_fingerprint):
            x = random.randint(0, 500) # x coord
            y = random.randint(0, 500) # y coord
            angle = random.randint(0, 360) # angle of minutiae
            minutiae_type = random.randint(0, 1) #0 for ridge, 1 for bif
            fingerprint.append((fingerprint_id, x, y, angle, minutiae_type)) # add to individual fingerprint data
        dataset.append(fingerprint)
    return dataset

def save_to_csv(dataset, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['ID', 'X', 'Y', 'Orientation', 'Type'])
        for fingerprint in dataset:
            for minutiae in fingerprint:
                writer.writerow(minutiae)

def encrypt_fingerprint_data(df, context):
    encrypted_minutiae_dict = {}
    for fingerprint_id in df['ID'].unique():
        fingerprint_minutiae = df[df['ID'] == fingerprint_id][['X', 'Y', 'Orientation', 'Type']].values.flatten().tolist()
        scaled_minutiae = [float(value) for value in fingerprint_minutiae]
        encrypted_minutiae = ts.ckks_vector(context, scaled_minutiae)
        encrypted_minutiae_dict[fingerprint_id]=encrypted_minutiae
    return encrypted_minutiae_dict

def find_matching_fingerprint_id(encrypted_query, encrypted_data, context):
    min_distance = float('inf')
    matching_id = None

    for fingerprint_id, encrypted_minutiae in encrypted_data.items():
        difference_vector = encrypted_query - encrypted_minutiae
        distance_squared = difference_vector.dot(difference_vector)

        distance_squared_decrypted = distance_squared.decrypt()
        distance = np.sqrt(distance_squared_decrypted)

        if distance < min_distance:
            min_distance = distance
            matching_id = fingerprint_id

    return matching_id

def normalize_and_scale_minutiae(df):
    # Example normalization - adjust based on your data's characteristics
    df['locX'] = (df['locX'] / df['locX'].max())
    df['locY'] = (df['locY'] / df['locY'].max())
    #df['Orientation'] = df['Orientation'] / 360
    # No change for 'Type' as it's categorical (0 or 1)
    return df

#### Generate Random Dataset of Fingerprint minutiae

In [115]:
df = pd.read_csv('fingerprint_test.csv')
df

print(df)

      ID  Type  LocationX  LocationY          Orientation
0      0     0         85        238             [-180.0]
1      0     0         87        273               [-0.0]
2      0     0         96        255               [-0.0]
3      0     0        103        293   [153.434948822922]
4      0     0        112        196  [-153.434948822922]
...   ..   ...        ...        ...                  ...
1395   6     1         -1         -1                  NaN
1396   6     1         -1         -1                  NaN
1397   6     1         -1         -1                  NaN
1398   6     1         -1         -1                  NaN
1399   6     1         -1         -1                  NaN

[1400 rows x 5 columns]


#### Setup TenSEAL context

In [182]:
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=16384*2,
    coeff_mod_bit_sizes=[60, 40, 40, 40, 60]
)

context.global_scale = 2**40
context.generate_galois_keys()
context.generate_relin_keys()
secret_key = context.secret_key()

In [194]:
fingerprint_id = 0
fingerprint_minutiae = df[df['ID'] == fingerprint_id][['LocationX', 'LocationY', 'Orientation', 'Type']].values.flatten().tolist()
print("Vector size: ", len(fingerprint_minutiae))
fingerprint_id = 0
fingerprint_minutiae2 = df[df['ID'] == fingerprint_id][['LocationX', 'LocationY', 'Orientation', 'Type']].values.flatten().tolist()
print("Vector size: ", len(fingerprint_minutiae2))

Vector size:  800
Vector size:  800


In [195]:
scaled_minutiae = []

for value in fingerprint_minutiae:
    try:
        # Attempt to convert directly to float
        converted_value = float(value)
        if np.isnan(converted_value):
            converted_value = 0  # Replace NaN with 0 or another appropriate value
        scaled_minutiae.append(converted_value)
    except ValueError:
        # If direct conversion fails, try evaluating as a list and then convert each element
        try:
            list_values = ast.literal_eval(value)
            if isinstance(list_values, list):
                scaled_list_values = [float(v) if not np.isnan(float(v)) else 0 for v in list_values]  # Replace NaN within the list
                scaled_minutiae.extend(scaled_list_values)
            else:
                raise ValueError
        except ValueError:
            # If conversion still fails, handle or report the erroneous value
            print(f"Could not convert value: {value}")

val = 0
target_length = 1000

while len(scaled_minutiae) < target_length:
    scaled_minutiae.append(val)
print(type(scaled_minutiae))
print("List size: ", len(scaled_minutiae))
arr1 = np.array(scaled_minutiae)

<class 'list'>
List size:  1000


In [196]:
scaled_minutiae2 = []

for value in fingerprint_minutiae2:
    try:
        # Attempt to convert directly to float
        converted_value = float(value)
        if np.isnan(converted_value):
            converted_value = 0  # Replace NaN with 0 or another appropriate value
        scaled_minutiae2.append(converted_value)
    except ValueError:
        # If direct conversion fails, try evaluating as a list and then convert each element
        try:
            list_values = ast.literal_eval(value)
            if isinstance(list_values, list):
                scaled_list_values = [float(v) if not np.isnan(float(v)) else 0 for v in list_values]  # Replace NaN within the list
                scaled_minutiae2.extend(scaled_list_values)
            else:
                raise ValueError
        except ValueError:
            # If conversion still fails, handle or report the erroneous value
            print(f"Could not convert value: {value}")

val = 0
target_length = 1000

while len(scaled_minutiae2) < target_length:
    scaled_minutiae2.append(val)
print(type(scaled_minutiae2))
print("List size: ", len(scaled_minutiae2))
arr2 = np.array(scaled_minutiae2)

<class 'list'>
List size:  1000


In [197]:
#arr1 = [1,2,3,4,5]
#arr2 = [6,7,8,9,10]
distance_squared = (arr1-arr2).dot(arr1-arr2) # dot product distance between two encrypted datapoints
print(distance_squared)
enc1 = ts.ckks_vector(context, arr1)
enc2 = ts.ckks_vector(context, arr2)


0.0


In [198]:


distance_squared = (enc1-enc2).dot(enc1-enc2) # dot product distance between two encrypted datapoints
distance_squared_decrypted = distance_squared.decrypt() # decrypt distance to test similarity (should be large value -> fail)

"""
distance_squared2 = (encrypted_minutiae-encrypted_minutiae3).dot(encrypted_minutiae-encrypted_minutiae3) # same as above
distance_squared_decrypted2 = distance_squared2.decrypt() # decrypt distance (should be very small value -> success)
"""
print("Squared Euclidean distance (decrypted for demo):", distance_squared_decrypted)
#print(distance_squared_decrypted2)

Squared Euclidean distance (decrypted for demo): [-3.52195903557116e-05]


#### test single print data against entire encrypted data and return ID (doesn't work 100% correctly yet)

In [114]:
id_match = find_matching_fingerprint_id(encrypted_minutiae, encrypted_data, context)
print(id_match)

C:\Users\quinn\AppData\Local\Temp\ipykernel_26180\3742005205.py:41: RuntimeWarning: invalid value encountered in sqrt
  distance = np.sqrt(distance_squared_decrypted)


3
